# SST2

In [12]:
import pandas as pd
import textflint
# load GLUE sst2 dataset and save splits as .csv
from datasets import load_dataset
dataset = load_dataset("glue", "sst2")
# rename "sentence" column to "x"
# rename "label" column to "y"
# replace 0 with negative and 1 with positive
dataset['train'] = dataset['train'].rename_column('sentence', 'x').rename_column('label', 'y')
dataset['validation'] = dataset['validation'].rename_column('sentence', 'x').rename_column('label', 'y')
dataset['test'] = dataset['test'].rename_column('sentence', 'x').rename_column('label', 'y')

def transform_labels(example):
    example["y"] = "negative" if example["y"] == 0 else "positive"
    return example

dataset['train'].to_csv('../data/GLUE_textflint/sst2_train.csv')
dataset['validation'].to_csv('../data/GLUE_textflint/sst2_val.csv')
dataset['test'].to_csv('../data/GLUE_textflint/sst2_test.csv')

Creating CSV from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

204793

now run textflint on the dataset  
`textflint --dataset sst2_val.csv --config config.json`  
config sth like  
```
{
  "task": "SA",
  "out_dir": "./DATA/",
  "trans_methods": [
    ["SpellingError", "Typos"]
  ]
}
```

In [19]:
def transform_back(file_path, out_path):
    import json
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Strip whitespace and parse the JSON object
            d = json.loads(line.strip())
            data.append(d)
            
    import pandas as pd
    df = pd.DataFrame(data)
    
    # rename "x" column to "sentence"
    # rename "y" column to "label"
    df = df.rename(columns={"x": "sentence", "y": "label"})
    df.to_csv(out_path)  # "../data/GLUE_textflint/sst2_dev_textflint.csv"

In [21]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/DATA_test/trans_Pipeline_SpellingError_Typos_random_1813.json', '../data/GLUE_textflint/sst2_test_textflint.csv')

In [22]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/DATA_train/trans_Pipeline_SpellingError_Typos_random_59223.json', '../data/GLUE_textflint/sst2_train_textflint.csv')

# QNLI

In [1]:
from datasets import load_dataset
dataset = load_dataset("glue", "qnli")
from datasets import Features, Value
# Cast the label column to a string feature
for split in dataset.keys():
    dataset[split] = dataset[split].cast(
        Features({**dataset[split].features, "label": Value("string")})
    )
    dataset[split] = dataset[split].map(
        lambda x: {"label": "entailment" if x["label"] == "0" else "neutral" if x["label"] == "1" else x["label"]}
    )
    # expects 'NLI': ['premise', 'hypothesis'], y instead of question, sentence, label
    dataset[split] = dataset[split].rename_column('question', 'premise').rename_column('sentence', 'hypothesis').rename_column('label', 'y')

    # remoe elements where y is -1
    dataset[split] = dataset[split].filter(lambda example: example["y"] != -1)

dataset['train'].to_csv('../data/GLUE_textflint/qnli_train.csv')
dataset['validation'].to_csv('../data/GLUE_textflint/qnli_val.csv')
dataset['test'].to_csv('../data/GLUE_textflint/qnli_test.csv')


Casting the dataset:   0%|          | 0/104743 [00:00<?, ? examples/s]

Map:   0%|          | 0/104743 [00:00<?, ? examples/s]

Filter:   0%|          | 0/104743 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/5463 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/105 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

1328272

In [4]:
def transform_back(file_path, out_path):
    import json
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Strip whitespace and parse the JSON object
            d = json.loads(line.strip())
            data.append(d)
            
    import pandas as pd
    df = pd.DataFrame(data)

    df = df.rename(columns={"premise": "question", "hypothesis": "sentence", "y": "label"})
    
    df['label'] = df['label'].apply(
        lambda x: '0' if x == 'entailment' else '1' if x == 'neutral' else x
    )
    
    df.to_csv(out_path)  # "../data/GLUE_textflint/sst2_dev_textflint.csv"

In [3]:
import pandas as pd
pd.DataFrame(dataset['validation'])

,premise,hypothesis,y,idx
0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment,0
1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,neutral,1
2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,neutral,2
3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment,3
4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",neutral,4
...,...,...,...,...
5458,How many people lived in Warsaw in 1939?,Unfortunately this belief still lives on in Po...,neutral,5458
5459,One of FIS' agenda items was to force women to...,Taking advantage of economic failure and unpop...,entailment,5459
5460,Dendritic cells are named that because they re...,Dendritic cells serve as a link between the bo...,neutral,5460
5461,When did it retract to a inland formation?,"However, the rainforest still managed to thriv...",neutral,5461


In [5]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QNLI/DATA_VAL/trans_Pipeline_SpellingError_Typos_random_5463.json','/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QNLI/qnli_val_textflint.csv')

In [7]:
transform_back(
    '/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QNLI/DATA_TRAIN/trans_Pipeline_SpellingError_Typos_random_104741.json',
    '/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QNLI/qnli_train_textflint.csv')

# MNLI

In [10]:
from datasets import load_dataset
dataset = load_dataset("glue", "mnli")
from datasets import Features, Value
# Cast the label column to a string feature
for split in dataset.keys():
    dataset[split] = dataset[split].cast(
        Features({**dataset[split].features, "label": Value("string")})
    )
    dataset[split] = dataset[split].map(
        lambda x: {
            "label":
                "entailment"
                if x["label"] == "0"
                else "contradiction"
                if x["label"] == "2"
                else "neutral"
                if x["label"] == "1"
                else x["label"]
        }
    )
    # expects 'NLI': ['premise', 'hypothesis'], y instead of question, sentence, label
    dataset[split] = dataset[split].rename_column('label', 'y')

    # remoe elements where y is -1
    dataset[split] = dataset[split].filter(lambda example: example["y"] != -1)

dataset['train'].to_csv('../data/GLUE_textflint/mnli_train.csv')
dataset['validation_matched'].to_csv('../data/GLUE_textflint/mnli_validation-matched.csv')
dataset['validation_mismatched'].to_csv('../data/GLUE_textflint/mnli_validation-mismatched.csv')

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

test_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

...

Filter:   0%|          | 0/9847 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/393 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

1944084

In [8]:
def transform_back(file_path, out_path):
    import json
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Strip whitespace and parse the JSON object
            d = json.loads(line.strip())
            data.append(d)

    import pandas as pd
    df = pd.DataFrame(data)

    df = df.rename(columns={"y": "label"})
    df['label'] = df['label'].apply(
        lambda x: '0' if x == 'entailment' else '2' if x == 'contradiction' else '1' if x == 'neutral' else x
    )

    df.to_csv(out_path)  # "../data/GLUE_textflint/sst2_dev_textflint.csv"

In [9]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/MNLI/DATA_TRAIN/trans_Pipeline_SpellingError_Typos_random_391466.json','/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/MNLI/mnli_train_textflint.csv')

In [10]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/MNLI/DATA_VAL-MATCHED/trans_Pipeline_SpellingError_Typos_random_9779.json','/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/MNLI/mnli_val_matched_textflint.csv')

In [11]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/MNLI/DATA_VAL-MISMATCHED/trans_Pipeline_SpellingError_Typos_random_9813.json','/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/MNLI/mnli_val_mismatched_textflint.csv')

# QQP

In [18]:
from datasets import load_dataset
dataset = load_dataset("glue", "qqp")
from datasets import Features, Value
# Cast the label column to a string feature
for split in dataset.keys():
    dataset[split] = dataset[split].cast(
        Features({**dataset[split].features, "label": Value("string")})
    )
    dataset[split] = dataset[split].map(
        lambda x: {
            "label":
                "neutral"
                if x["label"] == "0"
                else "entailment"
                if x["label"] == "1"
                else x["label"]
        }
    )
    # expects 'NLI': ['premise', 'hypothesis'], y instead of question, sentence, label
    dataset[split] = dataset[split].rename_column('label', 'y').rename_column('question1', 'premise').rename_column('question2', 'hypothesis')

    # remove elements where y is -1
    dataset[split] = dataset[split].filter(lambda example: example["y"] != -1)

dataset['train'].to_csv('../data/GLUE_textflint/qqp_train.csv')
dataset['validation'].to_csv('../data/GLUE_textflint/qqp_validation.csv')

train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

Filter:   0%|          | 0/390965 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/364 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

5553174

In [1]:
def transform_back(file_path, out_path):
    import json
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Strip whitespace and parse the JSON object
            d = json.loads(line.strip())
            data.append(d)

    import pandas as pd
    df = pd.DataFrame(data)

    df = df.rename(columns={"premise": "question1", "hypothesis": "question2", "y": "label"})
    
    df['label'] = df['label'].apply(
        lambda x: '0' if x == 'neutral' else '1' if x == 'entailment' else x
    )

    df.to_csv(out_path) 
    

In [2]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QQP/DATA_TRAIN/trans_Pipeline_SpellingError_Typos_random_363477.json','/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QQP/qqp_train_textflint.csv')

In [3]:
transform_back('/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QQP/DATA_VALIDATION/trans_Pipeline_SpellingError_Typos_random_40385.json','/Users/anna/Documents/git projects.nosync/StyleTokenizer/data/GLUE_textflint/QQP/qqp_val_textflint.csv')